In [1]:
from collections import defaultdict
from glob import glob
from random import choice, sample


from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D,Activation, Dropout, BatchNormalization,  GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Subtract,Multiply
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from skimage.io import imshow
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import cv2
import os
from tqdm import tqdm
from tensorflow.keras.applications.resnet50 import preprocess_input
from random import sample, choice

from tensorflow.keras import layers
from tensorflow.keras import preprocessing
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K

In [2]:
# File paths
train_file_path = "data/train/"
train_relationships_path = "../csv_files/train_relationships.csv"
#the validation set will be family members F09...
validation_set = "F09"

#get all the images
all_images = glob(train_file_path + "*/*/*.jpg")

#seperate the train and validation sets
train_images = [img for img in all_images if validation_set not in img]
val_images = [img for img in all_images if validation_set in img]
    

In [3]:
print(len(all_images))

12379


In [4]:
# train_person_to_images_map = defaultdict(list)

# ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

# for x in train_images:
#     train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

# val_person_to_images_map = defaultdict(list)

# for x in val_images:
#     val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

In [5]:
#create a dictionary with key=family member and value=list of pictures of family member
def fam_mem_pics(all_images):
    #create dictionary
    fam_member_dict = {}
    #create list for member pictures
    fam_member_lst = []
    i = 0
    while i < len(all_images)-1:
        #get the family member as key
        split_path = all_images[i].split('/')
        key = split_path[2]+'/'+split_path[3]
        #check if picture is about the same fmaily member
        same_member = True

        while same_member:
            #check if same family member
            if key in all_images[i]:
                #append the image to the family member list
                fam_member_lst.append(all_images[i])
                
            else:
                #changed family member
                same_member = False
            
            #update the counter to not go out of bounds
            if (i+1) < len(all_images):  
                i += 1
            else:
                #break if out of bounds
                break
                
        #check if key is already in dictionary - just in case the images are out of order
        if key in fam_member_dict.keys():
            #combine list
            fam_member_dict[key] = fam_member_dict[key] + fam_member_lst
        else: 
            #create key=family member and value= list of family member's pictures
            fam_member_dict[key] = fam_member_lst
        
        fam_member_lst = []
        
    #return the member_dictionary 
    return fam_member_dict


In [6]:
#get a family member image dictionary
train_person_to_images_map = fam_mem_pics(train_images)
val_person_to_images_map = fam_mem_pics(val_images)

In [7]:
# train_person_to_images_map

In [8]:
# val_person_to_images_map

In [9]:
#get the raltionships dataframe
df = pd.read_csv("csv_files/train_relationships.csv")

#get isolate the the family/member portions to compare
all_family_members = []
for img in all_images:
    split_path = img.split('/')
    all_family_members.append(split_path[2]+'/'+split_path[3])

#some relationships are not present within the relationship csv
#remove them by only keeping the ones mentioned in the dataset
df = df[df['p1'].isin(all_family_members)]
df = df[df['p2'].isin(all_family_members)]

#seperate the training and validation labels
train = df[~df['p1'].str.contains(validation_set)]
val = df[df['p1'].str.contains(validation_set)]

#only keep the values in the dictionary keys
train = train[train['p1'].isin(list(train_person_to_images_map.keys()))]
val = val[val['p1'].isin(list(val_person_to_images_map.keys()))]

train = train[train['p2'].isin(list(train_person_to_images_map.keys()))]
val = val[val['p2'].isin(list(val_person_to_images_map.keys()))]

#turn dataframes to tuples to get labels
train = list(zip(train['p1'].values, train['p2'].values))
val = list(zip(val['p1'].values, val['p2'].values))

In [10]:
print(len(train))
train[:10] #labels

2247


[('F0002/MID1', 'F0002/MID3'),
 ('F0002/MID2', 'F0002/MID3'),
 ('F0005/MID1', 'F0005/MID2'),
 ('F0005/MID3', 'F0005/MID2'),
 ('F0009/MID1', 'F0009/MID4'),
 ('F0009/MID1', 'F0009/MID3'),
 ('F0009/MID1', 'F0009/MID2'),
 ('F0009/MID1', 'F0009/MID6'),
 ('F0009/MID2', 'F0009/MID4'),
 ('F0009/MID2', 'F0009/MID6')]

In [11]:
def read_img(path):
    img = cv2.imread(path)
    img = np.array(img).astype(np.float)
    return preprocess_input(img)

#create a generator
def gen(family_member_labels, family_member_map, batch_size=16):
    while True:
        #sample from half of true labels - don't repeat grab
        half_batch_size = int(batch_size/2)
        batch_family_members = sample(family_member_labels, half_batch_size)
        #all these should be of label 1
        label = np.ones(len(batch_family_members))

        #grab data that isn't a combination from the labels set
        # create a set of size of 'batch size'
        fam_mem_keys = list(family_member_map.keys())
        while len(batch_family_members) < batch_size:
            #get random numbers to choose
            rand_num = np.random.randint(0, len(fam_mem_keys), 2)
            #get random family member
            fam_mem_1 = fam_mem_keys[rand_num[0]]
            fam_mem_2 = fam_mem_keys[rand_num[1]]

            #check if the two random people are family members - do not want that
            condition_1 = (fam_mem_1, fam_mem_2)
            condition_2 = (fam_mem_2, fam_mem_1)
            if fam_mem_1 not in fam_mem_2 and condition_1 not in family_member_labels and condition_2 not in family_member_labels:
                #add onto combination as not being family
                batch_family_members.append(condition_1)
                #add label as 0 for not family
                label = np.concatenate([label,[0]])

    #     print(batch_family_members)
    #     print(label)
    #     print('------------------')
#         X1 = [choice(family_member_map[x[0]]) for x in batch_family_members]
#         X1 = np.array([read_img(x) for x in X1])

#         X2 = [choice(family_member_map[x[1]]) for x in batch_family_members]
#         X2 = np.array([read_img(x) for x in X2])
        
        person_1_imgs = []
        person_2_imgs = []
        #get photos for each person
        for fam_mem_tup in batch_family_members:
            #peson 1's set of images
            mem_1_all_imgs = family_member_map[fam_mem_tup[0]]
            #person 2's set of images
            mem_2_all_imgs = family_member_map[fam_mem_tup[1]]

            #select random image
            person_1_choice = choice(mem_1_all_imgs)
            person_2_choice = choice(mem_2_all_imgs)
            
            #turn the images into arrays
            for person, pic_path in enumerate([person_1_choice, person_2_choice]):
                #read the image
                img = cv2.imread(pic_path)
                #turn to array
                img = np.array(img).astype(np.float)
                img_arr = preprocess_input(img)

                #add to lst
                if person == 0: #person 1
                    person_1_imgs.append(img_arr)
                else: #person 2
                    person_2_imgs.append(img_arr)

        #turn to numpy arrays
        person_1_imgs = np.asarray(person_1_imgs)
        person_2_imgs = np.asarray(person_2_imgs)
#         print(person_1_imgs.shape)
#         print(person_2_imgs.shape)
    
#         yield [X1, X2], label
        yield [person_1_imgs, person_2_imgs], label

In [12]:
gen(train, train_person_to_images_map, batch_size=16)

<generator object gen at 0x1379869a8>

In [13]:
from tensorflow.keras.applications.vgg19 import VGG19


# Define Model Architecture 
def siamese_model(pretrained_model=None):
    #left and right inputs for siamese network
    left_image = layers.Input(shape = (224, 224, 3))
    right_image = layers.Input(shape = (224, 224, 3))
    
    #start model
    model = models.Sequential()
    #check if there is a pretrained model
    if pretrained_model is not None:
        model.add(pretrained_model)
        
    model.add(layers.Dense(128, activation = "relu"))
    pretrained_model.trainable = False
    
    x1 = model(left_image)
    x2 = model(right_image)
    
    L2_normalized_layer_1 = layers.Lambda(lambda x: K.l2_normalize(x, axis = 1))
    X1_normal = L2_normalized_layer_1(x1)
    X2_normal = L2_normalized_layer_1(x2)

    L1_layer = layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([X1_normal, X2_normal])
    
    prediction = layers.Dense(1, activation = "sigmoid")(L1_distance)
    
    siamese_net = models.Model(inputs = [left_image, right_image], outputs = prediction)

    siamese_net.compile(loss = "binary_crossentropy", metrics = ["acc"], optimizer = optimizers.Adam(0.00001))
    
    return siamese_net

In [54]:
base_model = VGG19(weights='imagenet', include_top=False)
kinship_model = siamese_model(pretrained_model=base_model)
kinship_model.fit_generator(gen(train, train_person_to_images_map, batch_size = 16),
                    validation_data = gen(val, val_person_to_images_map, batch_size = 16), 
                    epochs = 100, verbose = 2, 
                    steps_per_epoch = 200, 
                    validation_steps = 100)

Epoch 1/100


ValueError: A target array with shape (16, 1) was passed for an output of shape (None, 7, 7, 1) while using as loss `binary_crossentropy`. This loss expects targets to have the same shape as the output.

# Test

In [37]:
from tqdm import tqdm

In [38]:
from tqdm import tqdm

test_path = "data/test/"

#return a set of inputerd size as generator
def gen_2(test_set, size=32):
    return (test_set[i:i + size] for i in range(0, len(test_set), size))

submission_df = pd.read_csv('csv_files/sample_submission.csv')

predictions = []


for batch in tqdm(gen_2(submission_df.img_pair.values)):
#     print(batch)
    img_1 = []
    img_2 = []
    #seperate image paths
    for pair_img in batch:
        pairs = pair_img.split('-')
        img_1.append(pairs[0])
        img_2.append(pairs[1])
    
    pic_1 = []
    pic_2 = []
    #read the image names
    for imge_1, imge_2 in zip(img_1, img_2):
        pic_1.append(read_img(test_path + imge_1))
        pic_2.append(read_img(test_path + imge_2))
    
    pic_1 = np.array(pic_1)
    pic_2 = np.array(pic_2)
#     print(pic_1)
#     print(pic_2)
    
    #predict using the test image arrays 
    pred = kinship_model.predict([pic_1, pic_2]).ravel().tolist()
    #combine list
    predictions += pred
    
submission_df['is_related'] = predictions



0it [00:00, ?it/s]

NameError: name 'kinship_model' is not defined

In [35]:
submission_

,img_pair,is_related
0,face05508.jpg-face01210.jpg,0
1,face05750.jpg-face00898.jpg,0
2,face05820.jpg-face03938.jpg,0
3,face02104.jpg-face01172.jpg,0
4,face02428.jpg-face05611.jpg,0
5,face01219.jpg-face00274.jpg,0
6,face04262.jpg-face00555.jpg,0
7,face03697.jpg-face01892.jpg,0
8,face03524.jpg-face00319.jpg,0
9,face03410.jpg-face05368.jpg,0


In [39]:
# from tqdm import tqdm

# test_path = "data/test/"

# def chunker(seq, size=32):
#     return (seq[pos:pos + size] for pos in range(0, len(seq), size))

# submission = pd.read_csv('csv_files/sample_submission.csv')

# predictions = []

# chunker(submission.img_pair.values)

# for batch in tqdm(chunker(submission.img_pair.values)):
# #     print(batch)
    
    
#     X1 = [x.split("-")[0] for x in batch]
#     X1 = np.array([read_img(test_path + x) for x in X1])

#     X2 = [x.split("-")[1] for x in batch]
#     X2 = np.array([read_img(test_path + x) for x in X2])

#     print(X1)
# #     print(X2)
    
    
# #     pred = kinship_model.predict([X1, X2]).ravel().tolist()
# #     predictions += pred

# # submission['is_related'] = predictions

# # submission.to_csv("vgg_face.csv", index=False)

In [ ]:
# kinship_model_json = kinship_model.to_json()
# with open("kinship_model.json", "w") as json_file:
#     json_file.write(kinship_model_json)
    
# kinship_model.save_weights("kinship_model_weights.h5")

In [219]:
# from tensorflow.keras.applications.vgg19 import VGG19

# def baseline_model():
#     input_1 = Input(shape=(224, 224, 3))
#     input_2 = Input(shape=(224, 224, 3))

# #     base_model = VGGFace(model='resnet50', include_top=False)
    
#     base_model = VGG19(weights='imagenet', include_top=False)
    
#     for x in base_model.layers[:-3]:
#         x.trainable = True

#     x1 = base_model(input_1)
#     x2 = base_model(input_2)

#     # x1_ = Reshape(target_shape=(7*7, 2048))(x1)
#     # x2_ = Reshape(target_shape=(7*7, 2048))(x2)
#     #
#     # x_dot = Dot(axes=[2, 2], normalize=True)([x1_, x2_])
#     # x_dot = Flatten()(x_dot)

#     x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
#     x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])

#     x3 = Subtract()([x1, x2])
#     x3 = Multiply()([x3, x3])

#     x = Multiply()([x1, x2])

#     x = Concatenate(axis=-1)([x, x3])

#     x = Dense(100, activation="relu")(x)
#     x = Dropout(0.01)(x)
#     out = Dense(1, activation="sigmoid")(x)

#     model = Model([input_1, input_2], out)

#     model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

# #     model.summary()

#     return model


In [37]:
# model = baseline_model()
# # model.load_weights(file_path)
# model.fit_generator(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
#                     validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=20, verbose=2,
#                     workers=4, steps_per_epoch=200, validation_steps=100)

In [206]:
# def read_img(path):
#     img = cv2.imread(path)
#     img = np.array(img).astype(np.float)
#     return preprocess_input(img)


# def gen(list_tuples, person_to_images_map, batch_size=16):
#     ppl = list(person_to_images_map.keys())
#     while True:
#         batch_tuples = sample(list_tuples, batch_size // 2)
#         labels = [1] * len(batch_tuples)
# #         print(batch_tuples)
# #         print(labels)
#         while len(batch_tuples) < batch_size:
#             p1 = choice(ppl)
#             p2 = choice(ppl)
# #             print('p1: ', p1)
# #             print('p2: ', p2)
#             if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
#                 batch_tuples.append((p1, p2))
#                 labels.append(0)
# #         print(batch_tuples)
# #         print(labels)
#         for x in batch_tuples:
# #             print('x:, ', x)
#             if not len(person_to_images_map[x[0]]):
# #                 print('x[0]:, ', x)
#                 print(x[0])
# #         print('------------------------')
#         X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
# #         print(len(X1))
# #         print(X1)
# #         print('------------------------')
#         X1 = np.array([read_img(x) for x in X1])
# #         print(X1)
# #         print('------------------------')
#         X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
#         X2 = np.array([read_img(x) for x in X2])
        
# #         print(type(X1))
# #         print(type(X2))
# #         print([X1,X2])
#         yield [X1, X2], labels


# gen(train, train_person_to_images_map, batch_size=16)

<generator object gen at 0x13db494f8>

In [ ]:
# gen(train, train_person_to_images_map, batch_size=16)

In [14]:
# def gen(list_tuples, person_to_images_map, batch_size=16):
#     ppl = list(person_to_images_map.keys())
#     while True:
#         batch_tuples = sample(list_tuples, batch_size // 2)
#         labels = [1] * len(batch_tuples)
#         while len(batch_tuples) < batch_size:
#             p1 = choice(ppl)
#             p2 = choice(ppl)

#             if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
#                 batch_tuples.append((p1, p2))
#                 labels.append(0)

#         for x in batch_tuples:
#             if not len(person_to_images_map[x[0]]):
#                 print(x[0])

#         X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
#         X1 = np.array([read_img(x) for x in X1])

#         X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
#         X2 = np.array([read_img(x) for x in X2])
        
# #         print(X1)
        
#         yield [X1, X2], labels
        
# # gen(train, train_person_to_images_map, batch_size=16)

In [ ]:
# pip install opencv-python